In [ ]:
pip install pandas


Imports

In [ ]:
import pandas as pd
import numpy as np
import sklearn 
import seaborn as sns

I load my database into a Dataframe

In [ ]:
df = pd.read_csv("creditcard.csv")
df.head()

If we read the documentation we can only know what are exactly 2 features: time and amount. Time is the tikme that passed between the nth transaction and the fisrt one. Amount is the quantity of money. We also have class (0 = nice transaction, 1 = fraud).

First we want to know some information of the dataframe:
- Missing values
- Entries
- Datatypes

In [ ]:
df.info()

As we can observe we have no missing values.

Now I want to study the distributions of our "class" feature

In [ ]:
sns.countplot(df, x = "Class")
df["Class"].value_counts()

Very unbalanced dataset, there are too much more of legit transactions. I create two subsets, separating them by the value on the feature Class

In [ ]:
legit = df[df["Class"]== 0]
fraud = df[df["Class"]== 1]
fraud.shape

I need to study the statistics of each subset


In [ ]:
print(legit["Amount"].describe())
print(fraud["Amount"].describe())

In [ ]:
df.groupby('Class').mean()

Now we need to balance the data. The technique I will use is undersampling. This is building a sample dataset with a new distribution. I will tage a random sample for the legit subset of 492 observations. That is the same quantity of fraudulent transactions that we have.

In [ ]:
legit_sample = legit.sample(n = 492)
legit_sample.describe()

In [ ]:
even_df = pd.concat([legit_sample, fraud], axis = 0)
even_df["Class"].value_counts()
even_df.groupby('Class').mean()

Now I start the process to split the data into training and testing samples.